Uses the cfgrib engine for xarraya to open the data.
Find cfgrib here. https://pypi.org/project/cfgrib/

In [ ]:
import numpy as np
import os
import pathlib
import scipy.interpolate
import xarray as xr
import xml.dom.minidom

In [ ]:
DATA_DIR = pathlib.Path(os.getenv('DATA_DIR'))
GRIB_FILE = DATA_DIR / 'data/test_download/gdps/2020112512/CMC_glb_RH_ISBL_925_latlon.15x.15_2020112512_P129.grib2'
SWOB_FILE = DATA_DIR / 'data/test_download/swob/20201125/CYUL/2020-11-25-1600-CYUL-MAN-swob.xml'

In [ ]:
d = xr.open_dataset(GRIB_FILE, engine='cfgrib')

In [ ]:
d

In [ ]:
d.r.plot()

In [ ]:
obs = xml.dom.minidom.parse(SWOB_FILE)

In [ ]:
observations = obs.getElementsByTagName('om:Observation')
observation = observations[0]

In [ ]:
id_elements = obs.getElementsByTagName('identification-elements')[0]

In [ ]:
metadata = {}
for c in id_elements.childNodes:
    if c.attributes['name'].value in ['lat', 'long']:
        metadata[c.attributes['name'].value] = float(c.attributes['value'].value)

        

In [ ]:
metadata

I could not get the interpolation to work directly on the data read by cfgrib. 
d.r.interp crashes.

In [ ]:
d.interp(longitude=metadata['long'], latitude=metadata['lat'])

# Multiple files

In [ ]:
temperature_files = DATA_DIR / 'data/test_download/gdps/2020112512'
temperature_files = sorted(list(temperature_files.glob('*TMP_ISBL_1000*_P00*.grib2')))

In [ ]:
print(temperature_files)

In [ ]:
d = xr.open_mfdataset(temperature_files, engine='cfgrib', concat_dim='step', combine='nested', parallel=True)

In [ ]:
d.interp(latitude=60.0, longitude=-74).t.compute()